# Token Broker via Azure Function (MPE Pattern)

This notebook demonstrates the **Hub & Spoke Token Broker** pattern for Fabric workspaces with **OAP (Outbound Access Policies)** enabled.

## Why This Pattern?

When OAP is enabled on your Fabric workspace, outbound calls to `login.microsoftonline.com` are **blocked**.  
The Function App acts as a secure token broker — Fabric reaches it via **MPE (Managed Private Endpoint)**, and the Function calls Entra ID on your behalf.

```
Fabric Notebook
  │   POST /api/GetSPToken  (via MPE — no direct Entra ID call needed)
  ▼
Azure Function ──→ login.microsoftonline.com  (gets SP token using MSAL)
  │
  └──→ returns { access_token, expires_in, scope, caller }
         │
Fabric uses token to connect to DB / REST API directly
```

## Supported Target Scopes

| Resource             | targetScope value                                        |
|----------------------|----------------------------------------------------------|
| Azure SQL / Synapse  | `https://database.windows.net/.default`                 |
| Azure Management API | `https://management.azure.com/.default`                 |
| Azure Storage        | `https://storage.azure.com/.default`                    |
| Power BI / Fabric    | `https://analysis.windows.net/powerbi/api/.default`     |
| Custom App           | `api://<your-app-client-id>/.default`                   |

## Setup (One-time)

1. Deploy the Function App to Azure
2. Configure a **Managed Private Endpoint** from your Fabric workspace to the Function App
3. Update the configuration values in the next cell


In [ ]:
from notebookutils import mssparkutils
import requests
import json

# ═══════════════════════════════════════════════════════════════
# CONFIGURATION - Update these values for your deployment
# ═══════════════════════════════════════════════════════════════

# Your Function App's Client ID (app registration in Entra ID)
# Used as the audience when acquiring the token to call the Function
FUNC_APP_CLIENT_ID = "1dbe7604-4c66-4d06-859e-06230744526c"

# Your Function App URL reachable from Fabric via MPE
# MPE private endpoint:  https://<private-fqdn>/api/GetSPToken
# Public endpoint:       https://<func-app-name>.azurewebsites.net/api/GetSPToken
FUNCTION_URL = "https://fabricmpeapis.azurewebsites.net/api/GetSPToken"

# Target scope — the resource you want the SP token FOR.
# The Function App calls login.microsoftonline.com on your behalf (OAP bypass).
#
# Azure SQL / Synapse Analytics:
TARGET_SCOPE = "https://database.windows.net/.default"
#
# Other examples (uncomment one):
# TARGET_SCOPE = "https://management.azure.com/.default"         # ARM / Azure REST
# TARGET_SCOPE = "https://storage.azure.com/.default"            # Azure Storage
# TARGET_SCOPE = "https://analysis.windows.net/powerbi/api/.default"  # Power BI / Fabric
# TARGET_SCOPE = "api://<your-app-client-id>/.default"           # Custom App

# ═══════════════════════════════════════════════════════════════

print("Step 1: Getting your identity token from Fabric (for calling the Function)...")

try:
    # Fabric sends YOUR identity (user or workspace MSI) to the Function for validation.
    # The audience MUST match the Function App's app registration App ID URI.
    audience = f"api://{FUNC_APP_CLIENT_ID}"
    fabric_token = mssparkutils.credentials.getToken(audience)
    print(f"  Token acquired ({len(fabric_token)} chars)")
    print(f"  Note: This token is for calling the Function — NOT the downstream DB token")
except Exception as e:
    print(f"  Failed to get token: {e}")
    print("  This must run inside a Fabric notebook session (not a local env or pipeline without MSI)")
    raise

print(f"\nStep 2: Calling Function to get SP token for scope: {TARGET_SCOPE}")
print(f"  POST {FUNCTION_URL}")
print(f"  (Function calls login.microsoftonline.com on your behalf — OAP bypass)")

try:
    headers = {
        "Authorization": f"Bearer {fabric_token}",
        "Content-Type": "application/json"
    }
    payload = {
        "targetScope": TARGET_SCOPE
    }

    response = requests.post(FUNCTION_URL, headers=headers, json=payload, timeout=30)
    print(f"  Response: HTTP {response.status_code}")

except requests.exceptions.ConnectionError as e:
    print(f"  Connection failed: {e}")
    print("  Check: Is the MPE configured and approved in your Fabric workspace?")
    raise
except Exception as e:
    print(f"  Error: {e}")
    raise

print("\nStep 3: Extracting SP token from response...")

if response.status_code == 200:
    body = response.json()
    sp_token       = body.get("access_token")
    caller         = body.get("caller", {})
    expires_in     = body.get("expires_in", 3600)

    print(f"\n  SUCCESS — SP token obtained for: {body.get('scope')}")
    print(f"  Token type:  {body.get('token_type')}")
    print(f"  Expires in:  {expires_in}s ({expires_in // 3600}h {(expires_in % 3600) // 60}m)")
    print(f"\n  Caller type:     {caller.get('type')}  ({'Interactive User' if caller.get('type') == 'USER' else 'Pipeline / Workspace MSI'})")
    print(f"  Caller identity: {caller.get('display')}")
    print(f"  Caller OID:      {caller.get('oid')}")
    print(f"\n  Token preview: {sp_token[:60]}...")
else:
    print(f"\n  Error ({response.status_code}):")
    try:
        print(json.dumps(response.json(), indent=2))
    except Exception:
        print(response.text)
    raise RuntimeError(f"Function returned HTTP {response.status_code}")


In [ ]:
import struct
import pyodbc

# ═══════════════════════════════════════════════════════════════
# USE SP TOKEN TO CONNECT TO AZURE SQL / SYNAPSE
# (Token was obtained by the Function — no login.microsoft.com call from Fabric)
# ═══════════════════════════════════════════════════════════════

SQL_SERVER   = "your-server.database.windows.net"   # e.g. myserver.database.windows.net
SQL_DATABASE = "your-database"

# Build the AAD token struct that pyodbc expects
# The token must be for audience: https://database.windows.net/
token_bytes   = sp_token.encode("utf-16-le")
token_struct  = struct.pack(f"<I{len(token_bytes)}s", len(token_bytes), token_bytes)
SQL_COPT_SS_ACCESS_TOKEN = 1256  # pyodbc constant for AAD token auth

conn_str = (
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={SQL_SERVER};"
    f"DATABASE={SQL_DATABASE};"
    f"Encrypt=yes;TrustServerCertificate=no;"
)

print(f"Connecting to {SQL_SERVER}/{SQL_DATABASE} using SP token...")

with pyodbc.connect(conn_str, attrs_before={SQL_COPT_SS_ACCESS_TOKEN: token_struct}) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT SYSTEM_USER, DB_NAME(), GETUTCDATE()")
    row = cursor.fetchone()
    print(f"\n  Connected as:  {row[0]}")
    print(f"  Database:      {row[1]}")
    print(f"  Server time:   {row[2]}")
    print("\n  SP token authentication successful!")


## Troubleshooting

| Issue | Cause | Fix |
|-------|-------|-----|
| `mssparkutils not found` | Not running in Fabric | Must run in a Fabric notebook session |
| `ConnectionError` to Function URL | MPE not configured or not approved | Create & approve MPE from Fabric workspace to Function App |
| `401 Unauthorized` from Function | Wrong audience | Check `FUNC_APP_CLIENT_ID` matches Function App's Entra app registration |
| `403 Forbidden` from Function | MSI OID not whitelisted | Add workspace MSI OID to `allowed-msi-oids` secret in Key Vault |
| `500 Internal Error` | Key Vault / MSAL issue | Check Function App logs; verify KV secrets are set |
| pyodbc token error | Wrong scope for SQL | Use `https://database.windows.net/.default` for Azure SQL/Synapse |
| Token expired on DB connect | Token stale | Re-run cell to get a fresh token from the Function |

### Key Insight: OAP Bypass

When **Outbound Access Policies** are enabled on your Fabric workspace:
- ❌ Fabric notebook cannot call `login.microsoftonline.com` directly  
- ✅ Fabric notebook calls the **Function App via MPE** (allowed)  
- ✅ Function App calls `login.microsoftonline.com` (outside Fabric network, unrestricted)  
- ✅ Fabric uses the returned SP token to connect directly to the DB (SQL is accessible via MPE/VNet)
